In [1]:
pip install torch transformers datasets sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import json
from sentence_transformers import SentenceTransformer, util

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Load dataset
with open("datasetDL.json", "r") as file:
    data = json.load(file)["intents"]

# Flatten dataset: pertanyaan dan respons
questions = []
responses = []
for intent in data:
    questions.extend(intent["text"])
    responses.extend(intent["responses"])

# Load model Sentence Transformer
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

FileNotFoundError: [Errno 2] No such file or directory: 'datasetDL.json'

In [5]:
def generate_response(user_input):
    """
    Generate chatbot response based on semantic similarity.
    """
    # Encode masukan pengguna
    user_embedding = model.encode(user_input, convert_to_tensor=True)

    # Encode semua pertanyaan di dataset
    question_embeddings = model.encode(questions, convert_to_tensor=True)

    # Hitung kesamaan kosinus antara masukan pengguna dan pertanyaan dataset
    similarities = util.cos_sim(user_embedding, question_embeddings)

    # Cari pertanyaan dengan kesamaan tertinggi
    max_idx = similarities.argmax().item()
    max_similarity = similarities[0, max_idx].item()

    # Threshold untuk memastikan relevansi
    if max_similarity > 0.7:  # Anda bisa sesuaikan nilai threshold ini
        return responses[max_idx]
    else:
        return "Maaf, saya tidak memahami pertanyaan Anda. Bisa Anda jelaskan lebih lanjut?"


In [6]:
print("Chatbot: Halo! Saya di sini untuk membantu Anda. Ketik 'keluar' untuk berhenti.")

while True:
    user_input = input("Anda: ")
    if user_input.lower() == "keluar":
        print("Chatbot: Terima kasih! Sampai jumpa.")
        break

    response = generate_response(user_input)
    print(f"Chatbot: {response}")


Chatbot: Halo! Saya di sini untuk membantu Anda. Ketik 'keluar' untuk berhenti.
Anda: 


NameError: name 'model' is not defined

In [ ]:
# Data uji: pasangan pertanyaan dan respons yang diharapkan
test_data = [
    {"input": "Apa itu HIV?", "expected": "Singkatan dari Human Immunodeficiency Virus, sebuah virus yang menyerang sistem kekebalan tubuh manusia."},
    {"input": "Apa itu AIDS?", "expected": "Singkatan dari Acquired Immune Deficiency Syndrome, kumpulan gejala klinis akibat penurunan sistem imun yang timbul akibat infeksi HIV."},
    {"input": "Bagaimana cara mencegah HIV?", "expected": "Maaf, saya tidak memahami pertanyaan Anda. Bisa Anda jelaskan lebih lanjut?"},
]


In [ ]:
def evaluate_chatbot(test_data):
    correct = 0
    total = len(test_data)

    for data in test_data:
        user_input = data["input"]
        expected_response = data["expected"]

        # Generate response dari chatbot
        actual_response = generate_response(user_input)

        # Bandingkan dengan respons yang diharapkan
        if actual_response == expected_response:
            correct += 1

    accuracy = correct / total
    print(f"Akurasi Chatbot: {accuracy:.2f}")


In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_chatbot_detailed(test_data):
    y_true = []  # Respons yang diharapkan
    y_pred = []  # Respons dari chatbot

    for data in test_data:
        user_input = data["input"]
        expected_response = data["expected"]
        actual_response = generate_response(user_input)

        # Tambahkan hasil ke daftar
        y_true.append(expected_response)
        y_pred.append(actual_response)

    # Tokenisasi respons untuk evaluasi
    y_true_tokens = [" ".join(y.split()) for y in y_true]
    y_pred_tokens = [" ".join(y.split()) for y in y_pred]

    # Hitung precision, recall, dan F1-score (berbasis token match)
    precision = precision_score(y_true_tokens, y_pred_tokens, average="micro")
    recall = recall_score(y_true_tokens, y_pred_tokens, average="micro")
    f1 = f1_score(y_true_tokens, y_pred_tokens, average="micro")

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")


In [8]:
# Evaluasi
evaluate_chatbot(test_data)
evaluate_chatbot_detailed(test_data)


NameError: name 'evaluate_chatbot' is not defined